
The goal is to detect whether a driver has closed eyes or open eyes

In [6]:
#installs:
!pip install dlib 
!pip install face_recognition 

ERROR: Invalid requirement: '#face'
ERROR: Invalid requirement: '#library'


In [2]:
from PIL import Image, ImageDraw # Python Imaging library
import face_recognition
import os

ModuleNotFoundError: No module named 'face_recognition'

In [ ]:
#in order to make sure we have the right size
#define a function to crop eyes images
def eye_cropper(folders):

  #Going through each image file 
  for folder in os.listdir(folders):
    for file in os.listdir(folders + '/' + folder):
      #Loading image files usinf facial recognition package
      image = face_recognition.load_image_file(folders + '/' + folder + file)
      #get the face landmarks coordinates
      face_landmarks_list = face_recognition.face_landmarks(image)
      #create a placeholder list for eye coordinates
      eyes=[]
      try:
        eyes.append(face_landmarks_list[0]['left_eye'])
        eyes.append(face_landmarks_list[0]['left_eye'])
      except:
        continue
      #get the max  and min coordinates of eye
      for eye in eyes :
        x_max = max([coordinate[0] for coordinate in eye])
        x_min = min([coordinate[0] for coordinate in eye])
        y_max = max([coordinate[1] for coordinate in eye])
        y_min = min([coordinate[1] for coordinate in eye])


        #get the range of x & y
        x_range = x_max - x_min
        y_range = y_max - y_min



        if x_range > y_range :
          right = round (.5*x_range) +  x_max
          left = x_min - round(.5*x_range)
          bottom = round(((right-left) - y_range))/2 + y_max
          top = y_min - round(((right-left) - y_range))/2
        else: 
          bottom = round (.5*y_range) +  y_max
          top = y_min - round(.5*y_range)
          right = round(((bottom-top) - x_range))/2 + x_max
          left = x_min - round(((bottom-top) - x_range))/2
        #crop original image using cushioned coords
        im = Image.open(folders + '/' + folder + '/' + file)
        im = im.crop((left, top, right , bottom))
        
        #resize image for input into our model
        im = im.resize((80,80))

        #save file to output folder

        im.save('CroppedEyesFolder')









In [4]:
#Prepare Image Data :
import cv2
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from keras.models import Sequential
from keras.layers import Dense,Flatten , Conv2D, MaxPooling2D,Dropout


ModuleNotFoundError: No module named 'cv2'

In [ ]:
# Importing images & resizing them:
def load_images_from_folder(folder, eyes =0):
  count = 0
  error_count = 0
  images = []
  for filename in os.listdir(folder):
    try :
      img = cv2.imread(os.path.join(folder,filename))
      img = cv2.resize(img,(80,80)) 
      images.append([img,eyes])
    except:
      error_count +=1
      print('ErrorCount = '+ str(error_count))
      continue
    count+=1
    if count % 500 ==0:
      print(('Successful Image Import Count = ' +str(count)))
  return images

In [ ]:
#LABELING IMAGES FOR OBJECT DETECTION:


In [ ]:
folder = "../Train_Data"
open_eyes = load_images_from_folder(folder, 0)

folder ="../Train_Data"
closed_eyes = load_images_from_folder(folder,1)
eyes = close_eyes + open_eyes

NameError: ignored

In [ ]:
x = []
y= []
for features, label in eyes :
  x.append(features)
  y.append(label)



X = np.array(X).reshape(-1, 80, 80, 3)
y = np.array(y)
X = X/255.0

X_train, X_test, y_train, y_test = train_test_split(X, y, stratify = y)

NameError: ignored

In [ ]:
#Creating the Model:
#Intantiate the model:
model =Sequential([Conv2D(
                filters = 32, # number of filters
                kernel_size = (3,3), # height/width of filter
                activation = 'relu', # activation function 
                input_shape = (80,80,3) # shape of input (image)
                ),
                Conv2D(
                filters = 32, # number of filters
                kernel_size = (3,3), # height/width of filter
                activation = 'relu' # activation function 
                ),
                Conv2D(
                filters = 32, # number of filters
                kernel_size = (3,3), # height/width of filter
                activation = 'relu' # activation function 
                ),
                MaxPooling2D(pool_size = (2,2)),
                Conv2D(
                filters = 32, # number of filters
                kernel_size = (3,3), # height/width of filter
                activation = 'relu' # activation function 
                ),
                Conv2D(
                filters = 32, # number of filters
                kernel_size = (3,3), # height/width of filter
                activation = 'relu' # activation function 
                ),
                MaxPooling2D(pool_size=(2,2)),
                Flatten(),
                Dense(256, activation='relu'),
                Dropout(0.3),
                Dense(128, activation='relu'),
                Dropout(0.3),
                Dense(64, activation='relu'),
                Dropout(0.3),
                Dense(1, activation = 'sigmoid')


                  
])

In [ ]:
# compiling the model
model.compile(loss='binary_crossentropy',
                optimizer='adam',
                metrics=[tf.keras.metrics.AUC(curve = 'PR')])


In [ ]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 78, 78, 32)        896       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 76, 76, 32)        9248      
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 74, 74, 32)        9248      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 37, 37, 32)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 35, 35, 32)        9248      
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 33, 33, 32)        9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 16, 16, 32)       

In [ ]:
# fitting the model
model.fit(X_train,
            y_train,
            batch_size=800,
            validation_data=(X_test, y_test),
            epochs=24)

# evaluate the model 
model.evaluate(X_test, y_test, verbose=1)

NameError: ignored

In [ ]:
model.save('DrowsinessDetectionModel.h5')